In [1]:
# Hopkins CV group
# face detection 
# eye detection 
# iris detection 
# Writen by Jinxin
# 2018.10.11
import numpy as np
import matplotlib
import cv2
from PIL import Image
import glob
# PCA 

In [11]:
# collect image data 
# process the PCA data 
def data_collection():
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    #eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    count = 0
    #glasses_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
    while(True):
        ret, frame = cap.read()
        # set the size of the frame
        ##########

        ##########
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, 1.06, 10)


        # detect the face on the left side 

        for (x,y,w,h) in faces:
            cv2.rectangle(gray,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            count = count+1
        
            dir  = "train_data/Jinxin/Jinxin_%d.jpg" % count
            cv2.imwrite(dir,roi_gray)

        cv2.imshow('frame',gray)

    # when we press q on the keyborad, quit the program
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()

In [12]:
# resize the data 
# read all the image in file 
def data_resize():
    image_list = []
    for filename in glob.glob('train_data/Jinxin/*.jpg'): #assuming gif
        im=Image.open(filename)
        image_list.append(im)
    # resize all the image and saved in Jinxin_resized
    count = 0
    for image in image_list:

        count = count+1
        image = np.array(image);
        resized_image = cv2.resize(image, (100, 100))
        resized_image = resized_image[5:95,5:95]
        #print (resized_image.shape)
        dir  = "train_data/Jinxin_resized/Jinxin_resized_%d.jpg" % count
        cv2.imwrite(dir,resized_image)


In [13]:
data_collection()
data_resize()

In [2]:
# PCA
def zeroMean(dataMat):          
    meanVal=np.mean(dataMat,axis=0)       
    newData=dataMat-meanVal    
    return newData,meanVal 
def pca(dataMat,n):    
    newData,meanVal=zeroMean(dataMat) 
    # print (newData.shape)
    covMat=np.cov(newData,rowvar=0)        
    eigVals,eigVects=np.linalg.eig(np.mat(covMat)) 
    eigValIndice=np.argsort(eigVals)        
    n_eigValIndice=eigValIndice[-1:-(n+1):-1]   
    n_eigVect=eigVects[:,n_eigValIndice]        
    lowDDataMat=newData*n_eigVect              
    reconMat=(lowDDataMat*n_eigVect.T)+meanVal   
    return lowDDataMat, reconMat

In [3]:
# do PCA and get eigen_face
# read the train data
image_list = []
for filename in glob.glob('train_data/Jinxin_resized/*.jpg'): #assuming gif
    im=Image.open(filename)
    image_list.append(im)
    
N = len(image_list)
im = np.array(im)
[H,L] = im.shape
index = 0
dataMat = []

for image in image_list:
    
    image = np.array(image)
    dataMat.append(np.reshape(image,(H*L)))
    index = index+1

dataMat = np.array(dataMat).T
n = 50
eigen_face, reconMat = pca(dataMat,n)

print (eigen_face.shape)
print (reconMat.shape)


(8100, 50)
(8100, 1041)


In [6]:
# PCA face detection in real time half and half 
up_threshold_Jinxin = 13002236.8621
down_threshold_Jinxin = 8866691.00754
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
#glasses_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
while(True):
    ret, frame = cap.read()
    # set the size of the frame
    ##########
    
    ##########
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    [H,L] = gray.shape
    gray1 = gray[0:H+1,0:np.int16(round(L/2.))]
    gray2 = gray[0:H+1,np.int16(round(L/2.)):L+1]
    
    faces1 = face_cascade.detectMultiScale(gray1, 1.1, 10)
    faces2 = face_cascade.detectMultiScale(gray2, 1.1, 10)
    
    # detect the face on the left side 
    for (x,y,w,h) in faces1:
        cv2.rectangle(gray1,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        resized_image = cv2.resize(roi_gray, (100, 100))
        resized_image = resized_image[5:95,5:95]
        resized_image_v = np.reshape(resized_image,(90*90,1))
        # do PCA
        Wt = (eigen_face.T)*resized_image_v
        Wt_norm=np.linalg.norm(Wt)
        if Wt_norm< up_threshold_Jinxin and Wt_norm>down_threshold_Jinxin:
            cv2.putText(gray1,'This is Jinxin!',(50,20),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,255),2)
            
        else:
            cv2.putText(gray1,"This is not Jinxin!", (50,20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2,(0,0,255),2)
        print (Wt_norm) 
        
        
        # we can also use eye detetion to creat eye ball control
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #glass = glasses_cascade.detectMultiScale(roi_gray)
        #for (ex,ey,ew,eh) in eyes:
        #    cv2.rectangle(roi_gray,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        #for (gx,gy,gw,gh) in glass:
        #    cv2.rectangle(roi_gray,(gx,gy),(gx+gw,gy+gh),(0,255,0),2)
        
        #
    cv2.imshow('frame1',gray1)
    # detect the face on the right side 
    for (x,y,w,h) in faces2:
        cv2.rectangle(gray2,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]    
        # do PCA
        resized_image = cv2.resize(roi_gray, (100, 100))
        resized_image = resized_image[5:95,5:95]
        resized_image_v = np.reshape(resized_image,(90*90,1))
        # get projection using eigen_face
        Wt = (eigen_face.T)*resized_image_v
        Wt_norm=np.linalg.norm(Wt)
        if Wt_norm< up_threshold_Jinxin and Wt_norm>down_threshold_Jinxin:
            cv2.putText(gray2,'This is Jinxin!',(50,20),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,255),2)
        else:
            cv2.putText(gray2,"This is not Jinxin!", (50,20), cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,255),2)
        print (Wt_norm) 
        
        
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #glass = glasses_cascade.detectMultiScale(roi_gray)
        #for (ex,ey,ew,eh) in eyes:
        #    cv2.rectangle(roi_gray,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        #for (gx,gy,gw,gh) in glass:
        #    cv2.rectangle(roi_gray,(gx,gy),(gx+gw,gy+gh),(0,255,0),2)  
    
    cv2.imshow('frame2',gray2)
    
# when we press q on the keyborad, quit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()

4231605.066664054
4536650.930495926
5509305.5002275985
5497671.38863867
5729630.445371167
5317527.8273976855
5564958.712264674
5050989.948380958
5199385.230989377
4610772.927526119
4254680.836848105
5401614.379663331
5289364.533133384
5356997.957960222
5564091.239597729
4807428.837992701
4266138.949260379
5615468.60371359
4381178.43825805
4822986.052405024
4914879.217432056
4845231.62069802
4888563.07086744
4865148.715223082
4479808.557767951
4638104.208323444
4430752.575669743
4879628.142994087
4542783.433500466
4895738.556988284
4550505.779801012
4339452.712133788
5099031.538725404
4583389.78017302
4461112.345383178
4417121.257693301
4092703.3288042997
4750379.228416912
4465607.968232878
4336674.089024872
4635678.984562439
4375179.9046036275
4500189.412081469
4383680.229853301
5241436.466404303
6316607.472156167
7221447.700169153
8920408.498558715
9805669.40346717
11124484.768130753
9189418.29781745
5674309.229217575
6290207.172705407
4868116.941562138
4346987.495286663
4619969.00445

In [22]:
# PCA face detection in real time realtime location of Jinxin
up_threshold_Jinxin = 101444517.34148009
down_threshold_Jinxin = 85583807.89230745
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
#glasses_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
while(True):
    ret, frame = cap.read()
    # set the size of the frame
    ##########
    
    ##########
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    [H,L] = gray.shape
    

    faces = face_cascade.detectMultiScale(gray, 1.1, 10)

    
    # detect the face on the left side 
    for (x,y,w,h) in faces:
        cv2.rectangle(gray,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        resized_image = cv2.resize(roi_gray, (100, 100))
        resized_image = resized_image[5:95,5:95]
        resized_image_v = np.reshape(resized_image,(90*90,1))
        # do PCA
        Wt = (eigen_face.T)*resized_image_v
        Wt_norm=np.linalg.norm(Wt)
        print (Wt_norm)
        if Wt_norm< up_threshold_Jinxin and Wt_norm>down_threshold_Jinxin:
            # publish the topic 
            location = (x+w)/2.
            # pubilsh the topic
            distance = w/100.
            cv2.putText(gray,'Found Jinxin!',(50,20),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,255),2)
            dir_location = 'loaction:%d distance %d'% (location, distance)
            cv2.putText(gray,dir_location,(50,150),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,255),2)
            
        else:
            cv2.putText(gray,"Jinxin not here!", (50,20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2,(0,0,255),2)
            
        
        
        # we can also use eye detetion to creat eye ball control
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #glass = glasses_cascade.detectMultiScale(roi_gray)
        #for (ex,ey,ew,eh) in eyes:
            #cv2.rectangle(roi_gray,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        #for (gx,gy,gw,gh) in glass:
        #    cv2.rectangle(roi_gray,(gx,gy),(gx+gw,gy+gh),(0,255,0),2)
        
        #
    cv2.imshow('frame1',gray)
    
    
# when we press q on the keyborad, quit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()

99661214.33152594
99760204.0207213
105222186.5308193
101940235.82613933
101740761.44999413
96959135.98146807
93574015.12504786
97038928.62436776
94590479.59273408
95625120.11451735
96359384.61123191
103060301.92881925
99009844.55386968
100864061.88617559
100781506.88791324
100089279.04623014
102480851.83496669
101425567.14378631
99566940.19186798
98808970.94166164
95735944.96450765
95939914.36751477
93587378.40904689
95572793.13939428
93844332.22243766
94286738.40188114
91585346.73929402
96389764.85063675
97045770.80814461
96732890.19185129
95044873.39953531
94951125.13244185
96108093.75590873
95789449.18925443
96863993.6611011
95062993.39087616
94853155.00048113
97997649.51391457
97712725.84498666
98177335.1321087
98953092.19525711
96433512.66614114
96156140.63889375
93861260.25522935
93342375.29864219
94653526.05321708
97039036.01338474
96552728.40992747
97684483.53344436
95354108.55869968
100228455.11321484
99350702.42143391
96718632.31860405
96069653.0854813
98993693.48019765
97632

95960123.55948038
96917669.33819103
94471027.32122995
96003134.50193052
96481874.17394198
93315998.89574069
88857710.5679311
52692685.25668487
71353092.83696973
49922075.15120118
56653837.362887174
31987480.270371348
69637225.43483119
67048904.946055785
56872421.776203856
62402093.128005736
68266324.15601732
72255424.52868117
73920602.62935926
83993732.48691228
86033884.05835141
87415499.29628234
85351055.27761583
89069485.73038091
89590883.91627339
90913505.89936674
90403570.98291086
90291427.93465053
82289402.45012283
64302945.8485992
48569371.12561282
74872121.1591562
102758471.38636494
98839788.36510691
94362278.97301643
100723369.16467887
98105670.92298122
94744135.14166525
93343754.36733986
92786967.8439169
92998386.26706927
93034331.08927143
96807785.13717833
94631020.17757489
96567864.84786429
99849084.68709846
96454286.61411338
96729230.09385237
97300246.23298262
97578770.55545557
101080594.83061764
99938687.52330577
106264459.3707249
71438456.78923075
84453031.99763304
868336

In [19]:
# eyeball and iris tracking 
import numpy as np
import cv2
import numpy as np
import matplotlib
import cv2
from PIL import Image
import glob

def sequential_label(binary_in):
    a = binary_in.shape
    a = np.array([int(v) for v in a])
    I = binary_in
    label = 0
    labeled_image = np.zeros((a[0],a[1]))
    record = []
    for i in range(1,a[0]):
        for j in range(1,a[1]):
            if I[i][j]!=0: 
                if labeled_image[i-1][j-1]!=0:
                    labeled_image[i][j] = labeled_image[i-1][j-1]

                elif labeled_image[i][j-1]==0 and labeled_image[i-1][j]==0:
                    label = label+1
                    labeled_image[i][j] = label
                elif labeled_image[i-1][j]!=0 and labeled_image[i][j-1]==0:
                    labeled_image[i][j] = labeled_image[i-1][j]

                elif labeled_image[i-1][j]==0 and labeled_image[i][j-1]!=0:
                    labeled_image[i][j] = labeled_image[i][j-1]    

                elif labeled_image[i-1][j]!=0 and labeled_image[i][j-1]!=0:

                    if labeled_image[i][j-1]==labeled_image[i-1][j]:
                        labeled_image[i][j] = labeled_image[i][j-1]

                    else: 
                        minValue = min(labeled_image[i][j-1],labeled_image[i-1][j])
                        maxValue = max(labeled_image[i][j-1],labeled_image[i-1][j])
                        labeled_image[i][j] = labeled_image[i][j-1]
                        # record equivalence of labels
                        labeled_image = np.where(labeled_image == minValue, maxValue, labeled_image)

    labelled_img = labeled_image
    # normalized
    
    max_labelled_img = labelled_img.max()
    labelled_img = (labelled_img*(255/max_labelled_img)).astype('uint8')
    
        
    return labelled_img

def moment_opencv(labelled_in):
    res = {}
    for label_idx in np.unique(labelled_in):
        if label_idx == 0:
            continue
        masked_in = (labelled_in==label_idx).astype(labelled_in.dtype)
        moments = cv2.moments(masked_in)
        res[label_idx] = [moments['m00'],moments['m01'],moments['m10'],
                          moments['m02'],moments['m11'],moments['m20'],
                          moments['mu02'],moments['mu11'],moments['mu20']]
    return res

def binarize(gray_in, threshold=128):
    #ret,thresh1 = cv2.threshold(img,threshold,255,cv.THRESH_BINARY)
    binary_image = np.where(gray_in>threshold,0,255).astype('uint8')
    return binary_image

def find_iris(eye_image):
    eye = np.array(eye_image)
    gray_eye = np.array(eye)
    [H,L] = eye_image.shape    
    scaler_x= H/40.
    scaler_y=L/40.
    gray_eye = cv2.resize(gray_eye,(40,40))
    #cv2.imshow('frame2',gray_eye)

    print (scaler_x,scaler_y)
    gray_eye = cv2.medianBlur(gray_eye, 5)
    gray_eye = binarize(gray_eye,50)
    label = sequential_label(gray_eye)
    cv2.imshow('frame3',label)
    if label.max() == 0:
        x_hat = 0
        y_hat = 0
        
    else:
        
        res = moment_opencv(label)
        list_A = []
        list_i = []
        count = 0
        for i in res:
            list_A.append(res[i][0])
            count = count+1
            list_i.append(i)
        index = np.where(list_A == max(list_A))
                
        index = np.array(index)       
        
    
        if index.shape == (1,0):
            index = 0
            
        ii = list_i[index]

        x_hat = round((float(res[ii][1])/float(res[ii][0]))*scaler_x)
        y_hat = round((float(res[ii][2])/float(res[ii][0]))*scaler_y)
        r = 5
        return x_hat,y_hat

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
cap = cv2.VideoCapture(0)
count = 0
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray,1.1,10)
        
        count_eye = 0
        for (ex,ey,ew,eh) in eyes:
            count_eye = count_eye +1
            if count_eye == 2:
                break
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            eye_img = roi_gray[ey:ey+eh,ex:ex+ew]
            #eye_color = roi_color[ey:ey+eh,ex:ex+ew]
            count= count +1
            eye_image = eye_img
            if eye_image is not None:
                try:
                    x_hat,y_hat = find_iris(eye_image)

                except:
                    print ('cant find iris!')
                else:                
                    r = 2
                    cv2.circle(roi_color, (ex+x_hat, ey+y_hat), r, (0, 255, 0), 2)

                cv2.imshow('frame2',eye_image)
            
            
    cv2.imshow('frame1',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


1.175 1.175
cant find iris!
1.2 1.2
cant find iris!
1.275 1.275
cant find iris!


C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in double_scalars
C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


1.175 1.175
cant find iris!
1.2 1.2
cant find iris!
1.2 1.2
1.225 1.225
1.175 1.175
1.2 1.2
cant find iris!
1.15 1.15
1.1 1.1
1.075 1.075
cant find iris!
1.175 1.175
cant find iris!
1.025 1.025
cant find iris!
0.975 0.975
cant find iris!
1.15 1.15
cant find iris!
1.05 1.05
cant find iris!
1.275 1.275
cant find iris!
1.075 1.075
cant find iris!
1.175 1.175
cant find iris!
1.175 1.175
cant find iris!
1.25 1.25
cant find iris!
1.075 1.075
cant find iris!
1.125 1.125
cant find iris!
1.15 1.15
cant find iris!
1.125 1.125
1.05 1.05
cant find iris!
1.0 1.0
cant find iris!
1.125 1.125
cant find iris!
1.125 1.125
1.15 1.15
cant find iris!
1.075 1.075
cant find iris!
1.05 1.05
cant find iris!
1.25 1.25
1.1 1.1
cant find iris!
1.05 1.05
1.05 1.05
1.1 1.1
1.125 1.125
1.075 1.075
1.175 1.175
1.0 1.0
1.175 1.175
1.15 1.15
1.075 1.075
1.025 1.025
1.1 1.1
1.05 1.05
1.125 1.125
1.075 1.075
1.125 1.125
1.075 1.075
1.1 1.1
1.175 1.175
1.125 1.125
1.15 1.15
0.7 0.7
1.125 1.125
1.075 1.075
1.1 1.1
1.25 1.2

In [9]:
# test find_iris use single image 
image_list = []
for filename in glob.glob('train_data/Jinxin_eye/*.jpg'): 
    im=Image.open(filename)
    image_list.append(im)
# resize all the image and saved in Jinxin_resized
for eye in image_list:
    eye = np.array(eye)
    gray_eye = np.array(eye)
    gray_eye = cv2.resize(gray_eye,(40,40))
    #cv2.imshow('frame2',gray_eye)
    gray_eye = cv2.medianBlur(gray_eye, 5)
    gray_eye = binarize(gray_eye,50)
    label = sequential_label(gray_eye)
    
    if label.max() == 0:
        print ('find nothing')
    else:
        res = moment_opencv(label)
        list_A = []
        list_i = []
        count = 0
        for i in res:
            list_A.append(res[i][0])
            count = count+1
            list_i.append(i)
        index = np.where(list_A == min(list_A))
                
        index = np.array(index)       
        
    
        if index.shape == (1,0):
            index = 0
            
        ii = list_i[index]

        x_hat = round(float(res[ii][1])/float(res[ii][0]))
        y_hat = round(float(res[ii][2])/float(res[ii][0]))
        r = 5
        cv2.circle(eye, (x_hat, y_hat), r, (0, 255, 0), 2)
    cv2.imshow('frame',eye)
    cv2.waitKey(0)
cv2.destroyAllWindows()
##################################### backup code Just in case 

C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in double_scalars
C:\anaconda2\envs\OpenCV\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in multiply


find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing
find nothing


In [43]:
# Image Segmentation with Watershed Algorithm
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

image_list = []
for filename in glob.glob('train_data/Jinxin_eye/*.jpg'): 
    im=Image.open(filename)
    image_list.append(im)
    
img = image_list[30];
img = np.array(img)
gray = img

img = cv2.imread('Screenshot1.png')
img = np.array(img)
gray = cv2.medianBlur(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),5)

ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)
# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]

cv2.imshow('frame',img)

cv2.waitKey(0)
cv2.destroyAllWindows()



In [22]:
# face and eye detection example code 

import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

img = cv2.imread('heat.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.1, 10)
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
# brigt object detection
'''
Author Jinxin Dong 
2018.8.6
'''
import cv2 
import numpy as np 
  
camera = cv2.VideoCapture(0) 
if (camera.isOpened()): 
    print('Open') 
else: 
    print('The camera havent been open') 
    
size = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)), 
    int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))) 
print('size:'+repr(size)) 
  
es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 4)) 
kernel = np.ones((5, 5), np.uint8) 
background = None
# test end 
while True: 

    grabbed, frame_lwpCV = camera.read() 
    gray_lwpCV = cv2.cvtColor(frame_lwpCV, cv2.COLOR_BGR2GRAY) 
    gray_lwpCV = cv2.GaussianBlur(gray_lwpCV, (21, 21), 0) 


    if  background is None: 
        background = gray_lwpCV 
        continue

    diff = cv2.absdiff(background, gray_lwpCV) 
    diff = cv2.threshold(diff, 150, 255, cv2.THRESH_BINARY)[1] 
    diff = cv2.dilate(diff, es, iterations=2) 

    image, contours, hierarchy = cv2.findContours(diff.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    for c in contours: 
        if cv2.contourArea(c) < 1500: 
            continue
        (x, y, w, h) = cv2.boundingRect(c) 
        cv2.rectangle(frame_lwpCV, (x, y), (x+w, y+h), (0, 255, 0), 2) 

        cv2.imshow('contours', frame_lwpCV) 
        cv2.imshow('dis', diff) 

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): 
        break
# When everything done, release the capture 
camera.release()  
cv2.destroyAllWindows()

Open


AttributeError: 'module' object has no attribute 'CAP_PROP_FRAME_WIDTH'

In [17]:
# using ios web camera (take ipad as a camera) 

import cv2


cam_url='http://10.0.0.67:8081/video'
print cam_url
vc=cv2.VideoCapture(cam_url)
if vc.isOpened(): 
    rval, frame = vc.read()
else:
    rval = False

rval = True
while rval:
#    frame=cv2.resize(frame,(100,100)) 
    cv2.imshow("webcam test", frame)

    rval, frame = vc.read()
    
    key = cv2.waitKey(20)
    if key == 27: # exit on ESC
        break
        
camera.release()  
cv2.destroyAllWindows()        
        

In [9]:
# using web camera 
import cv2
cv2.namedWindow("webcam test")
cam_url='http://10.0.0.67:8081/video'
print cam_url

# screenshot
import os
from PIL import Image
from PIL import ImageGrab
import cv2
x = 345
y = 281
m = 462
n = 327
 
k = 54
print '!'
while True:
    box = (x,y,m,n)
    
    img = ImageGrab.grab(box)
    cv2.imshow("webcam test", img)
    #img.show()
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): 
        break
cv2.destroyAllWindows()        
        


In [ ]:
# face detection small example
import cv2
import numpy as np 
# the path of the image
imagepath = r'./heat.jpg'
face_cascade = cv2.CascadeClassifier(r'./haarcascade_frontalface_default.xml') 
image = cv2.imread(imagepath)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
'''
faces = face_cascade.detectMultiScale(
    gray,
    scaleFactor = 1.15,
    minNeighbors = 5,
    minSize = (5,5),
    flags = cv2.cv.CV_HAAR_SCALE_IMAGE
)
'''
faces = face_cascade.detectMultiScale(gray,1.3,5);
for(x,y,w,h) in faces:
    # cv2.rectangle(image,(x,y),(x+w,y+w),(0,255,0),2)
    cv2.circle(image,(int(np.round((x+x+w)/2)),int(np.round((y+y+h)/2))),int(np.round(w/2)),(0,255,0),2)
    
cv2.imshow("Find Faces!",image)
cv2.waitKey(0)


In [33]:
# detection of eyes
import cv2
import numpy as np 
# the path of the image
imagepath = r'./detect_eye.jpg'
face_cascade = cv2.CascadeClassifier(r'./haarcascade_eye.xml') 
image = cv2.imread(imagepath)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray,1.3,5);
for(x,y,w,h) in faces:
    # cv2.rectangle(image,(x,y),(x+w,y+w),(0,255,0),2)
    cv2.circle(image,(int(np.round((x+x+w)/2)),int(np.round((y+y+h)/2))),int(np.round(w/2)),(0,255,0),2)
    
cv2.imshow("Find Faces!",image)
cv2.waitKey(0)

         